In [2]:
# sample code to fetch a list of restaurants using zomatopy

import pprint, json
import zomatopy
from rasa_sdk import Action


In [13]:
def chkcity(name):
    loc = name
        
    #Read city.txt file where all tier1 and tier 2 city names are listed.
    city_file = open("data/city.txt", "r")
    cities = []
    for line in city_file:
        stripped_line = line.strip()
        cities.append(stripped_line.lower())

    city_file.close()
        
    #If city name is not in the list then return a message.
    if loc.lower() not in cities:
        print("Sorry, we don't operate in this city. Can you please specify some other location")
    return

In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_sdk import Action
from rasa_sdk.events import SlotSet
import zomatopy
import json

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

#This class will search restaurants based on provided location, cuisine and price range.
class ActionSearchRestaurants(Action):
    def name(self):
        return 'action_search_restaurants'
        
    def run(self, dispatcher, tracker, domain): 
        #use zompato api key.
        config={"user_key":"bc4e4a608fc7c3eb63e9f45f80c01fa9"}
        zomato = zomatopy.initialize_app(config)
        
        #Set default flag for search results
        found=False
        
        #Get location, cuisine and price for search.
        loc = tracker.get_slot('location')
        cuisine = tracker.get_slot('cuisine')
        price = tracker.get_slot('price')

        #If mandatory fields not passed then display error the message.
        if loc is None:
            response= "Sorry, we don't operate in this city.Please provide Tier-1/Tier-2 City name "+"\n"
            found=False
            dispatcher.utter_message(response)
            return False
        
        elif cuisine is None:
            response= "Please provide available Cuisine types"+"\n"
            found=False
            dispatcher.utter_message(response)
            return False
            
        elif price is None:
            response= "Please provide Budget criteria "+"\n"
            found=False
            dispatcher.utter_message(response)
            return False
        
        #Prepare the dictionary for cuisine and price
        cuisines_dict={'chinese':25,'italian':55,'north indian':50,'south indian':85,'american':1,'mexican':73}
        price_dict = {'low':1,'medium':2,'high':3}
        
        #prepare and Pass the required parameters to zomato restaurant_search method.
        location_detail=zomato.get_location(loc, 1)
        d1 = json.loads(location_detail)
        lat=d1["location_suggestions"][0]["latitude"]
        lon=d1["location_suggestions"][0]["longitude"]
        
        results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 50000)
        
        d = json.loads(results)
        count = 0
        response="Showing you top rated"+cuisine.capitalize()+" restaurants in "+loc.capitalize()+":\n"
                
        #If no resoults found from api call display the message.
        if d['results_found'] == 0:
            response= "No restaurant found for your criteria"+"\n"
            found=False
            dispatcher.utter_message(response)
            
        #if results found get top 10 restaurants with top rating first.
        else:           
            for restaurant in sorted(d['restaurants'],
                                     key=lambda x: x['restaurant']['user_rating']['aggregate_rating']
                                     , reverse=True): 
                
                #assign values to variables for reusability.
                cost_for_2=restaurant['restaurant']['average_cost_for_two']
                rstrnt_nm=restaurant['restaurant']['name']
                rstrnt_addr=restaurant['restaurant']['location']['address']
                rstrnt_rtng=restaurant['restaurant']['user_rating']['aggregate_rating']
                
                #If price range is low, get top 10 restaurants from this loop
                if((price_dict.get(price) == 1) and (int(cost_for_2) < 300) and (count < 10)):
                    response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                    response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                    count = count + 1
                    
                #If price range is medium, get top 10 restaurants from this loop        
                elif((price_dict.get(price) == 2) and (int(cost_for_2) >= 300) and (int(cost_for_2) <= 700) and (count < 10)):
                    response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                    response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                    count = count + 1      
                    
                #If price range is high, get top 10 restaurants from this loop     
                elif((price_dict.get(price) == 3) and (int(cost_for_2) > 700) and (count < 10)):
                    response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                    response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                    count = count + 1 
                    
                #Once count is 5, send the message to rasa with results.
                if(count==5):
                    dispatcher.utter_message(response)
                    found=True
                    
        #if the overall count is between 0 and 5, return the results to rasa
        if(count<5 and count>0):
            dispatcher.utter_message(response)
            found=True
        
        #if no results found for given price range then, return this message to rasa
        if(count==0):
            response = "Sorry, No results found for your criteria. Would you like to search for some other restaurant criteria?"+"\n"
            found=False
            dispatcher.utter_message(response)
        
        #Set emailbody content with top 10 results for future use if user opts for email option.
        return [SlotSet('emailbody',response),SlotSet('rest_found',found)]

#This class will send mail to given user email id.
class ActionSendEmail(Action):

    def name(self):
        return 'action_sendemail'

    def run(self, dispatcher, tracker, domain):
        
        #Config username and password for chatbot to send mails to user.
        from_user = 'baalebail.udemy@gmail.com'
        to_user = tracker.get_slot('email')
        password = 'Epsilon19'
        
        #smpt configuration for gmail.
        server = smtplib.SMTP('smtp.gmail.com',587)
        server.starttls()
        server.login(from_user, password)
        
        #Construct email subject and content
        subject = 'Top rated restaurants matching your search criteria'
        body_start="Hi there! Here are the " + subject + "." + "\n" + "\n"
        body_end="\n" + "\n"+ "Thank you for using Foodie service !"
        body_main=tracker.get_slot('emailbody')
        
        if body_main is not null:
            msg = MIMEMultipart()
            msg['From'] = from_user
            msg['TO'] = to_user
            msg['Subject'] = subject
            body = body_start + body_main + body_end
            msg.attach(MIMEText(body,'plain'))
            text = msg.as_string()
            server.sendmail(from_user,to_user,text)
        else:
            response="No data found"
            dispatcher.utter_message(response)
            
        server.close()

#This class will be used for checking if provided cuisine name belongs to listed cuisines.
class ActionCheckCuisine(Action):

    def name(self):
        return 'action_chkcuisine'

    def run(self, dispatcher, tracker, domain):
        cusn = tracker.get_slot('cuisine')
        found=False

        #Read cuisine.txt file where all cuisine names are listed.
        cuisine_file = open("data/cuisine.txt", "r")
        cuisines = []
        for line in cuisine_file:
            stripped_line = line.strip()
            cuisines.append(stripped_line.lower())

        cuisine_file.close()
        
        #If cuisine name is not in the list then return a message.
        if cusn.lower() not in cuisines:
            
            found=False
            response = "Can you please pick from available options for cuisine"+"\n"
            dispatcher.utter_message(response)
        else:
            found=True
        return [SlotSet('cusn_avl',found)]

#This class will be used for checking if provided city name belongs to Tier-1 and Tier-2 or not.
class ActionCheckLocation(Action):

    def name(self):
        return 'action_chklocation'

    def run(self, dispatcher, tracker, domain):
        loc = tracker.get_slot('location')
        found=False

        #Read city.txt file where all tier1 and tier 2 city names are listed.
        city_file = open("data/city.txt", "r")
        cities = []
        for line in city_file:
            stripped_line = line.strip()
            cities.append(stripped_line.lower())

        city_file.close()
        
        #If city name is not in the list then return a message.
        if loc.lower() not in cities:
            
            found=False
            response = " Sorry, we don’t operate in this city. Can you please specify some other location"+"\n"
            dispatcher.utter_message(response)
        else:
            found=True
        return [SlotSet('loc_avl',found)]

In [6]:
# specify location and cuisine
loc = 'Purulia'
cuisine = 'Mexican'
price = 'high'

#Tag price to a segment.
if price.isnumeric():
    if int(price) < 300:
        price='low'
    elif int(price) >= 300 and int(price) <= 700:
                price='medium'
    else:
        price='high'

#If mandatory fields not passed then display error the message.
if loc is None:
    response= "Sorry, we don't operate in this city.Please provide Tier-1/Tier-2 City name "+"\n"
    found=False
    print(response)
    #return False
        
elif cuisine is None:
    response= "Please provide available Cuisine types"+"\n"
    found=False
    print(response)
    #return False
            
elif price is None:
    response= "Please provide Budget criteria "+"\n"
    found=False
    print(response)
    #return False
        
# provide API key and initialise a 'zomato app' object
config={ "user_key": "bc4e4a608fc7c3eb63e9f45f80c01fa9"}
zomato = zomatopy.initialize_app(config)

# get_location gets the lat-long coordinates of 'loc'
location_detail=zomato.get_location(loc, 1)

# store retrieved data as a dict
d1 = json.loads(location_detail)

# separate lat-long coordinates
lat=d1["location_suggestions"][0]["latitude"]
lon=d1["location_suggestions"][0]["longitude"]

# cuisines code (used by zomatopy)
cuisines_dict=={'chinese':25,'italian':55,'north indian':50,'south indian':85,'american':1,'mexican':73}

#price dictionary
price_dict = {'low':1,'medium':2,'high':3}

print(str(cuisines_dict.get(cuisine)))
print(str(price_dict.get(price)))

# fetch and print results
results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 5)
d = json.loads(results)
#pprint.pprint(d)

None
3


In [7]:
count = 0
response="Showing you top rated "+cuisine.capitalize()+" restaurants in "+loc.capitalize()+":\n"
                
#If no resoults found from api call display the message.
if d['results_found'] == 0:
    response= "No restaurant found for your criteria"+"\n"
    found=False
    print(response)
            
#if results found get top 10 restaurants with top rating first.
else:           
    for restaurant in sorted(d['restaurants'],
                                key=lambda x: x['restaurant']['user_rating']['aggregate_rating']
                                , reverse=True): 
                
        #assign values to variables for reusability.
        cost_for_2=restaurant['restaurant']['average_cost_for_two']
        rstrnt_nm=restaurant['restaurant']['name']
        rstrnt_addr=restaurant['restaurant']['location']['address']
        rstrnt_rtng=restaurant['restaurant']['user_rating']['aggregate_rating']
                
        #If price range is low, get top 10 restaurants from this loop
        if((price_dict.get(price) == 1) and (int(cost_for_2) < 300) and (count < 10)):
            response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
            response=response+" and the average price for two people is: Rs."+ str(cost_for_2) +"\n"
            count = count + 1
                    
            #If price range is medium, get top 10 restaurants from this loop        
        elif((price_dict.get(price) == 2) and (int(cost_for_2) >= 300) and (int(cost_for_2) <= 700) and (count < 10)):
            response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
            response=response+" And the average price for two people is: Rs."+ str(cost_for_2) +"\n"
            count = count + 1      
                    
        #If price range is high, get top 10 restaurants from this loop     
        elif((price_dict.get(price) == 3) and (int(cost_for_2) > 700) and (count < 10)):
            response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
            response=response+" And the average price for two people is: Rs."+ str(cost_for_2) +"\n"
            count = count + 1 
                    
        #Once count is 5, send the message to rasa with results.
        if(count==5):
            print(response)
            found=True
                    
#if the overall count is between 0 and 5, return the results to rasa
if(count<5 and count>0):
    print(response)
    found=True
        
#if no results found for given price range then, return this message to rasa
if(count==0):
    response = "Sorry, No results found for your criteria. Would you like to search for some other restaurant criteria?"+"\n"
    found=False
    print(response)

Sorry, No results found for your criteria. Would you like to search for some other restaurant criteria?



In [16]:

    if d['results_found'] == 0:
        response= "No restaurant found for your criteria"+"\n"
        found=False
        print(response)
            
    #if results found get top 10 restaurants with top rating first.
    else:
        count=0
        response=""
        for restaurant in sorted(d['restaurants'],
                                     key=lambda x: x['restaurant']['user_rating']['aggregate_rating']
                                     , reverse=True):
            #assign values to variables for reusability.
            cost_for_2=restaurant['restaurant']['average_cost_for_two']
            rstrnt_nm=restaurant['restaurant']['name']
            rstrnt_addr=restaurant['restaurant']['location']['address']
            rstrnt_rtng=restaurant['restaurant']['user_rating']['aggregate_rating']
            
            #If price range is low, get top 10 restaurants from this loop
            if((price_dict.get(price) == 1) and (int(cost_for_2) < 300) and (count < 10)):
                response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                count = count + 1
                    
            #If price range is medium, get top 10 restaurants from this loop        
            elif((price_dict.get(price) == 2) and (int(cost_for_2) >= 300) and (int(cost_for_2) <= 700) and (count < 10)):
                response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                count = count + 1      
                    
            #If price range is high, get top 10 restaurants from this loop     
            elif((price_dict.get(price) == 3) and (int(cost_for_2) > 700) and (count < 10)):
                response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                count = count + 1 
                    
                #Once count is 5, send the message to rasa with results.
                if(count==5):
                    
                    print(response)
                    found=True
                    
        #if the overall count is between 0 and 5, return the results to rasa
        if(count<5 and count>0):
            print(response)
            found=True
        
        #if no results found for given price range then, return this message to rasa
        if(count==0):
            response = "Sorry, No results found for your criteria. Would you like to search for some other restaurant criteria?"+"\n"
            found=False
            print(response)

Joker's Kitchen in 94/1, Door 211/1, TTK Road, Near Ethiraj Kalayana Mandapam, Alwarpet, Chennai has been rated 4.3. And the average price for two people is: 800
Asvah 24 in 24 Beach Road, Sri Kapaleeshwar Nagar, Neelangarai, Chennai has been rated 4.2. And the average price for two people is: 1700
Pedreno's in 113/114, PS Sivaswami Salai, Mylapore, Chennai has been rated 4.1. And the average price for two people is: 1200
Tuscana Pizzeria in 19, 3rd Street, Wallace Garden Road, Nungambakkam, Chennai has been rated 4.1. And the average price for two people is: 1200
Tryst Cafe in 4/359, Gatsby Village, East Coast Road, Neelangarai, Chennai has been rated 4.1. And the average price for two people is: 800



In [ ]:
import json
import logging
import re
from typing import Any, Awaitable, Callable, Dict, List, Optional, Text

from rasa.core.channels.channel import InputChannel, OutputChannel, UserMessage
from rasa.utils.common import raise_warning
from sanic import Blueprint, response
from sanic.request import Request
from sanic.response import HTTPResponse
from slack import WebClient

logger = logging.getLogger(__name__)


class SlackBot(OutputChannel):
    """A Slack communication channel"""

    @classmethod
    def name(cls) -> Text:
        return "slack"

    def __init__(self, token: Text, slack_channel: Optional[Text] = None) -> None:

        self.slack_channel = slack_channel
        self.client = WebClient(token, run_async=True)
        super().__init__()

    @staticmethod
    def _get_text_from_slack_buttons(buttons: List[Dict]) -> Text:
        return "".join([b.get("title", "") for b in buttons])

    async def send_text_message(
        self, recipient_id: Text, text: Text, **kwargs: Any
    ) -> None:
        recipient = self.slack_channel or recipient_id
        for message_part in text.strip().split("\n\n"):
            await self.client.chat_postMessage(
                channel=recipient, as_user=True, text=message_part, type="mrkdwn",
            )

    async def send_image_url(
        self, recipient_id: Text, image: Text, **kwargs: Any
    ) -> None:
        recipient = self.slack_channel or recipient_id
        image_block = {"type": "image", "image_url": image, "alt_text": image}
        await self.client.chat_postMessage(
            channel=recipient, as_user=True, text=image, blocks=[image_block],
        )

    async def send_attachment(
        self, recipient_id: Text, attachment: Dict[Text, Any], **kwargs: Any
    ) -> None:
        recipient = self.slack_channel or recipient_id
        await self.client.chat_postMessage(
            channel=recipient, as_user=True, attachments=[attachment], **kwargs,
        )

    async def send_text_with_buttons(
        self,
        recipient_id: Text,
        text: Text,
        buttons: List[Dict[Text, Any]],
        **kwargs: Any,
    ) -> None:
        recipient = self.slack_channel or recipient_id

        text_block = {"type": "section", "text": {"type": "plain_text", "text": text}}

        if len(buttons) > 5:
            raise_warning(
                "Slack API currently allows only up to 5 buttons. "
                "Since you added more than 5, slack will ignore all of them."
            )
            return await self.send_text_message(recipient, text, **kwargs)

        button_block = {"type": "actions", "elements": []}
        for button in buttons:
            button_block["elements"].append(
                {
                    "type": "button",
                    "text": {"type": "plain_text", "text": button["title"]},
                    "value": button["payload"],
                }
            )
        await self.client.chat_postMessage(
            channel=recipient,
            as_user=True,
            text=text,
            blocks=[text_block, button_block],
        )

    async def send_custom_json(
        self, recipient_id: Text, json_message: Dict[Text, Any], **kwargs: Any
    ) -> None:
        json_message.setdefault("channel", self.slack_channel or recipient_id)
        json_message.setdefault("as_user", True)
        await self.client.chat_postMessage(**json_message)


class SlackInput(InputChannel):
    """Slack input channel implementation. Based on the HTTPInputChannel."""

    @classmethod
    def name(cls) -> Text:
        return "slack"

    @classmethod
    def from_credentials(cls, credentials: Optional[Dict[Text, Any]]) -> InputChannel:
        if not credentials:
            cls.raise_missing_credentials_exception()

        # pytype: disable=attribute-error
        return cls(
            credentials.get("slack_token"),
            credentials.get("slack_channel"),
            credentials.get("slack_retry_reason_header", "x-slack-retry-reason"),
            credentials.get("slack_retry_number_header", "x-slack-retry-num"),
            credentials.get("errors_ignore_retry", None),
        )
        # pytype: enable=attribute-error

    def __init__(
        self,
        slack_token: Text,
        slack_channel: Optional[Text] = None,
        slack_retry_reason_header: Optional[Text] = None,
        slack_retry_number_header: Optional[Text] = None,
        errors_ignore_retry: Optional[List[Text]] = None,
    ) -> None:
        """Create a Slack input channel.
        Needs a couple of settings to properly authenticate and validate
        messages. Details to setup:
        https://github.com/slackapi/python-slackclient
        Args:
            slack_token: Your Slack Authentication token. You can create a
                Slack app and get your Bot User OAuth Access Token
                `here <https://api.slack.com/slack-apps>`_.
            slack_channel: the string identifier for a channel to which
                the bot posts, or channel name (e.g. '#bot-test')
                If not set, messages will be sent back
                to the "App" DM channel of your bot's name.
            slack_retry_reason_header: Slack HTTP header name indicating reason that slack send retry request.
            slack_retry_number_header: Slack HTTP header name indicating the attempt number
            errors_ignore_retry: Any error codes given by Slack
                included in this list will be ignored.
                Error codes are listed
                `here <https://api.slack.com/events-api#errors>`_.
        """
        self.slack_token = slack_token
        self.slack_channel = slack_channel
        self.errors_ignore_retry = errors_ignore_retry or ("http_timeout",)
        self.retry_reason_header = slack_retry_reason_header
        self.retry_num_header = slack_retry_number_header

    @staticmethod
    def _is_app_mention(slack_event: Dict) -> bool:
        try:
            return slack_event["event"]["type"] == "app_mention"
        except KeyError:
            return False

    @staticmethod
    def _is_direct_message(slack_event: Dict) -> bool:
        try:
            return slack_event["event"]["channel_type"] == "im"
        except KeyError:
            return False

    @staticmethod
    def _is_user_message(slack_event: Dict) -> bool:
        return (
            slack_event.get("event")
            and (
                slack_event.get("event", {}).get("type") == "message"
                or slack_event.get("event", {}).get("type") == "app_mention"
            )
            and slack_event.get("event", {}).get("text")
            and not slack_event.get("event", {}).get("bot_id")
        )

    @staticmethod
    def _sanitize_user_message(text, uids_to_remove) -> Text:
        """Remove superfluous/wrong/problematic tokens from a message.
        Probably a good starting point for pre-formatting of user-provided text
        to make NLU's life easier in case they go funky to the power of extreme
        In the current state will just drop self-mentions of bot itself
        Args:
            text: raw message as sent from slack
            uids_to_remove: a list of user ids to remove from the content
        Returns:
            str: parsed and cleaned version of the input text
        """

        for uid_to_remove in uids_to_remove:
            # heuristic to format majority cases OK
            # can be adjusted to taste later if needed,
            # but is a good first approximation
            for regex, replacement in [
                (fr"<@{uid_to_remove}>\s", ""),
                (fr"\s<@{uid_to_remove}>", ""),  # a bit arbitrary but probably OK
                (fr"<@{uid_to_remove}>", " "),
            ]:
                text = re.sub(regex, replacement, text)

        """Find multiple mailto or http links like <mailto:xyz@rasa.com|xyz@rasa.com> or '<http://url.com|url.com>in text and substitute it with original content
        """

        pattern = r"(\<(?:mailto|http|https):\/\/.*?\|.*?\>)"
        match = re.findall(pattern, text)

        if match:
            for remove in match:
                replacement = remove.split("|")[1]
                replacement = replacement.replace(">", "")
                text = text.replace(remove, replacement)
        return text.strip()

    @staticmethod
    def _is_interactive_message(payload: Dict) -> bool:
        """Check wheter the input is a supported interactive input type."""

        supported = [
            "button",
            "select",
            "static_select",
            "external_select",
            "conversations_select",
            "users_select",
            "channels_select",
            "overflow",
            "datepicker",
        ]
        if payload.get("actions"):
            action_type = payload["actions"][0].get("type")
            if action_type in supported:
                return True
            elif action_type:
                logger.warning(
                    "Received input from a Slack interactive component of type "
                    f"'{payload['actions'][0]['type']}', for which payload parsing is not yet supported."
                )
        return False

    @staticmethod
    def _get_interactive_response(action: Dict) -> Optional[Text]:
        """Parse the payload for the response value."""

        if action["type"] == "button":
            return action.get("value")
        elif action["type"] == "select":
            return action.get("selected_options", [{}])[0].get("value")
        elif action["type"] == "static_select":
            return action.get("selected_option", {}).get("value")
        elif action["type"] == "external_select":
            return action.get("selected_option", {}).get("value")
        elif action["type"] == "conversations_select":
            return action.get("selected_conversation")
        elif action["type"] == "users_select":
            return action.get("selected_user")
        elif action["type"] == "channels_select":
            return action.get("selected_channel")
        elif action["type"] == "overflow":
            return action.get("selected_option", {}).get("value")
        elif action["type"] == "datepicker":
            return action.get("selected_date")

    async def process_message(
        self,
        request: Request,
        on_new_message: Callable[[UserMessage], Awaitable[Any]],
        text,
        sender_id: Optional[Text],
        metadata: Optional[Dict],
    ) -> Any:
        """Slack retries to post messages up to 3 times based on
        failure conditions defined here:
        https://api.slack.com/events-api#failure_conditions
        """
        retry_reason = request.headers.get(self.retry_reason_header)
        retry_count = request.headers.get(self.retry_num_header)
        if retry_count and retry_reason in self.errors_ignore_retry:
            logger.warning(
                f"Received retry #{retry_count} request from slack"
                f" due to {retry_reason}."
            )

            return response.text(None, status=201, headers={"X-Slack-No-Retry": 1})

        if metadata is not None:
            output_channel = metadata.get("out_channel")
        else:
            output_channel = None

        try:
            user_msg = UserMessage(
                text,
                self.get_output_channel(output_channel),
                sender_id,
                input_channel=self.name(),
                metadata=metadata,
            )

            await on_new_message(user_msg)
        except Exception as e:
            logger.error(f"Exception when trying to handle message.{e}")
            logger.error(str(e), exc_info=True)

        return response.text("")

    def get_metadata1(self, request: Request) -> Dict[Text, Any]:
        """Extracts the metadata from a slack API event (https://api.slack.com/types/event).
        Args:
            request: A `Request` object that contains a slack API event in the body.
        Returns:
            Metadata extracted from the sent event payload. This includes the output channel for the response,
            and users that have installed the bot.
        """
        slack_event = request.json
        event = slack_event.get("event", {})

        return {
            "out_channel": event.get("channel"),
            "users": slack_event.get("authed_users"),
        }
    
    def get_metadata(self, request: Request) -> Dict[Text, Any]:

        content_type = request.headers.get("content-type")

        # Slack API sends either a JSON-encoded or a URL-encoded body depending on the content

        if content_type == "application/json":

            # if JSON-encoded message is received

            slack_event = request.json

            event = slack_event.get("event", {})

            thread_id = event.get("thread_ts", event.get("ts"))

            users = (

                slack_event.get("authed_users", [])

                if "authed_users" in slack_event

                else slack_event.get("authorizations")[0].get("user_id")

            )

            return {

                "out_channel": event.get("channel"),

                "thread_id": thread_id,

                "users": users,

            }

        if content_type == "application/x-www-form-urlencoded":

            # if URL-encoded message is received

            output = request.form

            payload = json.loads(output["payload"][0])

            message = payload.get("message", {})

            thread_id = message.get("thread_ts", message.get("ts"))

            return {

                "out_channel": payload.get("channel", {}).get("id"),

                "thread_id": thread_id,

                "users": payload.get("user", {}).get("id"),

            }

        return {}

    
    def blueprint(
        self, on_new_message: Callable[[UserMessage], Awaitable[Any]]
    ) -> Blueprint:
        slack_webhook = Blueprint("slack_webhook", __name__)

        @slack_webhook.route("/", methods=["GET"])
        async def health(_: Request) -> HTTPResponse:
            return response.json({"status": "ok"})

        @slack_webhook.route("/webhook", methods=["GET", "POST"])
        async def webhook(request: Request) -> HTTPResponse:
            if request.form:
                output = request.form
                payload = json.loads(output["payload"][0])

                if self._is_interactive_message(payload):
                    sender_id = payload["user"]["id"]
                    text = self._get_interactive_response(payload["actions"][0])
                    if text is not None:
                        metadata = self.get_metadata(request)
                        return await self.process_message(
                            request, on_new_message, text, sender_id, metadata
                        )
                    elif payload["actions"][0]["type"] == "button":
                        # link buttons don't have "value", don't send their clicks to bot
                        return response.text("User clicked link button")
                return response.text(
                    "The input message could not be processed.", status=500
                )

            elif request.json:
                output = request.json
                event = output.get("event", {})
                user_message = event.get("text", "")
                sender_id = event.get("user", "")
                metadata = self.get_metadata(request)

                if "challenge" in output:
                    return response.json(output.get("challenge"))

                elif self._is_user_message(output) and self._is_supported_channel(
                    output, metadata
                ):
                    return await self.process_message(
                        request,
                        on_new_message,
                        text=self._sanitize_user_message(
                            user_message, metadata["users"]
                        ),
                        sender_id=sender_id,
                        metadata=metadata,
                    )
                else:
                    logger.warning(
                        f"Received message on unsupported channel: {metadata['out_channel']}"
                    )

            return response.text("Bot message delivered.")

        return slack_webhook

    def _is_supported_channel(self, slack_event: Dict, metadata: Dict) -> bool:
        return (
            self._is_direct_message(slack_event)
            or self._is_app_mention(slack_event)
            or metadata["out_channel"] == self.slack_channel
        )

    def get_output_channel(self, channel: Optional[Text] = None) -> OutputChannel:
        channel = channel or self.slack_channel
        return SlackBot(self.slack_token, channel)

    def set_output_channel(self, channel: Text) -> None:
        self.slack_channel = channel


In [5]:
pprint.pprint(d)

{'restaurants': [{'restaurant': {'R': {'has_menu_status': {'delivery': 1,
                                                           'takeaway': -1},
                                       'is_grocery_store': False,
                                       'res_id': 51705},
                                 'all_reviews': {'reviews': [{'review': []},
                                                             {'review': []},
                                                             {'review': []},
                                                             {'review': []},
                                                             {'review': []}]},
                                 'all_reviews_count': 6908,
                                 'apikey': 'bc4e4a608fc7c3eb63e9f45f80c01fa9',
                                 'average_cost_for_two': 2000,
                                 'book_again_url': '',
                                 'book_form_web_view_url': '',
                    

In [14]:
chkcity('bangalore')

Sorry, we don't operate in this city. Can you please specify some other location


In [12]:
city_file = open("data/city.txt", "r")
loc='Bengaluru'
cities = []
for line in city_file:
  stripped_line = line.strip()
  cities.append(stripped_line.lower())

city_file.close()
print(cities)

if loc.lower() not in cities:
    print("city not found")


['ahmedabad', 'bengaluru', 'chennai', 'delhi', 'hyderabad', 'kolkata', 'mumbai', 'trichy', 'jaipur', 'coimbatore', 'agra', 'ajmer', 'aligarh', 'amravati', 'amritsar', 'asansol', 'aurangabad', 'bareilly', 'belgaum', 'bhavnagar', 'bhiwandi', 'bhopal', 'bhubaneswar', 'bikaner', 'bilaspur', 'bokaro steel city', 'chandigarh', 'coimbatore', 'cuttack', 'dehradun', 'dhanbad', 'bhilai', 'durgapur', 'dindigul', 'erode', 'faridabad', 'firozabad', 'ghaziabad', 'gorakhpur', 'gulbarga', 'guntur', 'gwalior', 'gurgaon', 'guwahati', 'hamirpur', 'hubli', 'dharwad', 'indore', 'jabalpur', 'jaipur', 'jalandhar', 'jammu', 'jamnagar', 'jamshedpur', 'jhansi', 'jodhpur', 'kakinada', 'kannur', 'kanpur', 'karnal', 'kochi', 'kolhapur', 'kollam', 'kozhikode', 'kurnool', 'ludhiana', 'lucknow', 'madurai', 'malappuram', 'mathura', 'mangalore', 'meerut', 'moradabad', 'mysore', 'nagpur', 'nanded', 'nashik', 'nellore', 'noida', 'patna', 'pondicherry', 'purulia', 'prayagraj', 'raipur', 'rajkot', 'rajahmundry', 'ranchi', 

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_sdk import Action
from rasa_sdk.events import SlotSet
import zomatopy
import json

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def search_restaurant(location,cuisin,price): 
    #use zompato api key.
    config={"user_key":"bc4e4a608fc7c3eb63e9f45f80c01fa9"}
    zomato = zomatopy.initialize_app(config)
        
    #Get location, cuisine and price for search.
    loc = location
    cuisine = cuisin
    price = price
        
    #Prepare the dictionary for cuisine and price
    cuisines_dict={'chinese':25,'italian':55,'north indian':50,'south indian':85,'american':1,'mexican':73}
    price_dict = {'low':1,'medium':2,'high':3}
        
    #prepare and Pass the required parameters to zomato restaurant_search method.
    location_detail=zomato.get_location(loc, 1)
    d1 = json.loads(location_detail)
    lat=d1["location_suggestions"][0]["latitude"]
    lon=d1["location_suggestions"][0]["longitude"]
        
    results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 50000)
        
    d = json.loads(results)
    count = 0
    response="Showing you top rated restaurants:"+"\n"
        
    #If no resoults found from api call display the message.
    if d['results_found'] == 0:
        response= "No restaurant found for your criteria"
        dispatcher.utter_message(response)
            
    #if results found get top 10 restaurants with top rating first.
    else:           
        for restaurant in sorted(d['restaurants'],
                                    key=lambda x: x['restaurant']['user_rating']['aggregate_rating']
                                    , reverse=True): 
                
            #assign values to variables for reusability.
            cost_for_2=restaurant['restaurant']['average_cost_for_two']
            rstrnt_nm=restaurant['restaurant']['name']
            rstrnt_addr=restaurant['restaurant']['location']['address']
            rstrnt_rtng=restaurant['restaurant']['user_rating']['aggregate_rating']
            print(cost_for_2,rstrnt_nm,rstrnt_addr,rstrnt_rtng)
                
            #If price range is low, get top 10 restaurants from this loop
            if((price_dict.get(price) == 1) and (int(cost_for_2) < 300) and (count < 10)):
                response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                count = count + 1
                    
            #If price range is medium, get top 10 restaurants from this loop        
            elif((price_dict.get(price) == 2) and (int(cost_for_2) >= 300) and (int(cost_for_2) <= 700) and (count < 10)):
                response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                count = count + 1      
                    
            #If price range is high, get top 10 restaurants from this loop     
            elif((price_dict.get(price) == 3) and (int(cost_for_2) > 700) and (count < 10)):
                response=response+ rstrnt_nm + " in "+ rstrnt_addr + " has been rated "+ rstrnt_rtng +"."
                response=response+" And the average price for two people is: "+ str(cost_for_2) +"\n"
                count = count + 1 
                    
            #Once count is 5, send the message to rasa with results.
            if(count==5):
                print(response)
                    
    #if the overall count is between 0 and 5, return the results to rasa
    if(count<5 and count>0):
        print(response)
        
     #if no results found for given price range then, return this message to rasa
    if(count==0):
        response = "Sorry, No results found for your criteria. Would you like to search for some other restaurants?"
        print(response)

In [23]:
search_restaurant('bangalore','chinese','high')

2000 Yauatcha Level 5, 1 MG Lido Mall, MG Road, Bangalore 4.7
1200 The Fatty Bao - Asian Gastro Bar 610, 3rd Floor, 12th Main, Off 80 Feet Road, Indiranagar, Bangalore 4.6
1100 Hae Kum Gang 20, Pauls Castle, Castle Street, Ashok Nagar, Brigade Road, Bangalore 4.6
2600 High Ultra Lounge 26/1, 31st Floor, Dr. Rajkumar Road, World Trade Centre, Brigade Gateway Campus, Malleshwaram, Bangalore 4.4
1100 China Pearl 53/1, 5th Cross, Koramangala 6th Block, Bangalore 4.4
Showing you top rated restaurants:
Yauatcha in Level 5, 1 MG Lido Mall, MG Road, Bangalore has been rated 4.7. And the average price for two people is: 2000
The Fatty Bao - Asian Gastro Bar in 610, 3rd Floor, 12th Main, Off 80 Feet Road, Indiranagar, Bangalore has been rated 4.6. And the average price for two people is: 1200
Hae Kum Gang in 20, Pauls Castle, Castle Street, Ashok Nagar, Brigade Road, Bangalore has been rated 4.6. And the average price for two people is: 1100
High Ultra Lounge in 26/1, 31st Floor, Dr. Rajkumar Ro

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging

from rasa_core.agent import Agent
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RegexInterpreter
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter

logger = logging.getLogger(__name__)


def run_weather_online(input_channel, interpreter,
                          domain_file="weather_domain.yml",
                          training_data_file='data/stories.md'):
    agent = Agent(domain_file,
                  policies=[MemoizationPolicy(), KerasPolicy()],
                  interpreter=interpreter)

    agent.train_online(training_data_file,
                       input_channel=input_channel,
                       max_history=2,
                       batch_size=50,
                       epochs=200,
                       max_training_samples=300)

    return agent


if __name__ == '__main__':
    logging.basicConfig(level="INFO")
    nlu_interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
    run_weather_online(ConsoleInputChannel(), nlu_interpreter)

In [ ]:
import json
import logging
import re
from typing import Any, Awaitable, Callable, Dict, List, Optional, Text

from rasa.core.channels.channel import InputChannel, OutputChannel, UserMessage
from rasa.utils.common import raise_warning
from sanic import Blueprint, response
from sanic.request import Request
from sanic.response import HTTPResponse
from slack import WebClient

logger = logging.getLogger(__name__)


class SlackBot(OutputChannel):
    """A Slack communication channel"""

    @classmethod
    def name(cls) -> Text:
        return "slack"

    def __init__(self, token: Text, slack_channel: Optional[Text] = None) -> None:

        self.slack_channel = slack_channel
        self.client = WebClient(token, run_async=True)
        super().__init__()

    @staticmethod
    def _get_text_from_slack_buttons(buttons: List[Dict]) -> Text:
        return "".join([b.get("title", "") for b in buttons])

    async def send_text_message(
        self, recipient_id: Text, text: Text, **kwargs: Any
    ) -> None:
        recipient = self.slack_channel or recipient_id
        for message_part in text.strip().split("\n\n"):
            await self.client.chat_postMessage(
                channel=recipient, as_user=True, text=message_part, type="mrkdwn",
            )

    async def send_image_url(
        self, recipient_id: Text, image: Text, **kwargs: Any
    ) -> None:
        recipient = self.slack_channel or recipient_id
        image_block = {"type": "image", "image_url": image, "alt_text": image}
        await self.client.chat_postMessage(
            channel=recipient, as_user=True, text=image, blocks=[image_block],
        )

    async def send_attachment(
        self, recipient_id: Text, attachment: Dict[Text, Any], **kwargs: Any
    ) -> None:
        recipient = self.slack_channel or recipient_id
        await self.client.chat_postMessage(
            channel=recipient, as_user=True, attachments=[attachment], **kwargs,
        )

    async def send_text_with_buttons(
        self,
        recipient_id: Text,
        text: Text,
        buttons: List[Dict[Text, Any]],
        **kwargs: Any,
    ) -> None:
        recipient = self.slack_channel or recipient_id

        text_block = {"type": "section", "text": {"type": "plain_text", "text": text}}

        if len(buttons) > 5:
            raise_warning(
                "Slack API currently allows only up to 5 buttons. "
                "Since you added more than 5, slack will ignore all of them."
            )
            return await self.send_text_message(recipient, text, **kwargs)

        button_block = {"type": "actions", "elements": []}
        for button in buttons:
            button_block["elements"].append(
                {
                    "type": "button",
                    "text": {"type": "plain_text", "text": button["title"]},
                    "value": button["payload"],
                }
            )
        await self.client.chat_postMessage(
            channel=recipient,
            as_user=True,
            text=text,
            blocks=[text_block, button_block],
        )

    async def send_custom_json(
        self, recipient_id: Text, json_message: Dict[Text, Any], **kwargs: Any
    ) -> None:
        json_message.setdefault("channel", self.slack_channel or recipient_id)
        json_message.setdefault("as_user", True)
        await self.client.chat_postMessage(**json_message)


class SlackInput(InputChannel):
    """Slack input channel implementation. Based on the HTTPInputChannel."""

    @classmethod
    def name(cls) -> Text:
        return "slack"

    @classmethod
    def from_credentials(cls, credentials: Optional[Dict[Text, Any]]) -> InputChannel:
        if not credentials:
            cls.raise_missing_credentials_exception()

        # pytype: disable=attribute-error
        return cls(
            credentials.get("slack_token"),
            credentials.get("slack_channel"),
            credentials.get("slack_retry_reason_header", "x-slack-retry-reason"),
            credentials.get("slack_retry_number_header", "x-slack-retry-num"),
            credentials.get("errors_ignore_retry", None),
        )
        # pytype: enable=attribute-error

    def __init__(
        self,
        slack_token: Text,
        slack_channel: Optional[Text] = None,
        slack_retry_reason_header: Optional[Text] = None,
        slack_retry_number_header: Optional[Text] = None,
        errors_ignore_retry: Optional[List[Text]] = None,
    ) -> None:
        """Create a Slack input channel.

        Needs a couple of settings to properly authenticate and validate
        messages. Details to setup:

        https://github.com/slackapi/python-slackclient

        Args:
            slack_token: Your Slack Authentication token. You can create a
                Slack app and get your Bot User OAuth Access Token
                `here <https://api.slack.com/slack-apps>`_.
            slack_channel: the string identifier for a channel to which
                the bot posts, or channel name (e.g. '#bot-test')
                If not set, messages will be sent back
                to the "App" DM channel of your bot's name.
            slack_retry_reason_header: Slack HTTP header name indicating reason that slack send retry request.
            slack_retry_number_header: Slack HTTP header name indicating the attempt number
            errors_ignore_retry: Any error codes given by Slack
                included in this list will be ignored.
                Error codes are listed
                `here <https://api.slack.com/events-api#errors>`_.

        """
        self.slack_token = slack_token
        self.slack_channel = slack_channel
        self.errors_ignore_retry = errors_ignore_retry or ("http_timeout",)
        self.retry_reason_header = slack_retry_reason_header
        self.retry_num_header = slack_retry_number_header

    @staticmethod
    def _is_app_mention(slack_event: Dict) -> bool:
        try:
            return slack_event["event"]["type"] == "app_mention"
        except KeyError:
            return False

    @staticmethod
    def _is_direct_message(slack_event: Dict) -> bool:
        try:
            return slack_event["event"]["channel_type"] == "im"
        except KeyError:
            return False

    @staticmethod
    def _is_user_message(slack_event: Dict) -> bool:
        return (
            slack_event.get("event")
            and (
                slack_event.get("event", {}).get("type") == "message"
                or slack_event.get("event", {}).get("type") == "app_mention"
            )
            and slack_event.get("event", {}).get("text")
            and not slack_event.get("event", {}).get("bot_id")
        )

    @staticmethod
    def _sanitize_user_message(text, uids_to_remove) -> Text:
        """Remove superfluous/wrong/problematic tokens from a message.

        Probably a good starting point for pre-formatting of user-provided text
        to make NLU's life easier in case they go funky to the power of extreme

        In the current state will just drop self-mentions of bot itself

        Args:
            text: raw message as sent from slack
            uids_to_remove: a list of user ids to remove from the content

        Returns:
            str: parsed and cleaned version of the input text
        """

        for uid_to_remove in uids_to_remove:
            # heuristic to format majority cases OK
            # can be adjusted to taste later if needed,
            # but is a good first approximation
            for regex, replacement in [
                (fr"<@{uid_to_remove}>\s", ""),
                (fr"\s<@{uid_to_remove}>", ""),  # a bit arbitrary but probably OK
                (fr"<@{uid_to_remove}>", " "),
            ]:
                text = re.sub(regex, replacement, text)

        """Find multiple mailto or http links like <mailto:xyz@rasa.com|xyz@rasa.com> or '<http://url.com|url.com>in text and substitute it with original content
        """

        pattern = r"(\<(?:mailto|http|https):\/\/.*?\|.*?\>)"
        match = re.findall(pattern, text)

        if match:
            for remove in match:
                replacement = remove.split("|")[1]
                replacement = replacement.replace(">", "")
                text = text.replace(remove, replacement)
        return text.strip()

    @staticmethod
    def _is_interactive_message(payload: Dict) -> bool:
        """Check wheter the input is a supported interactive input type."""

        supported = [
            "button",
            "select",
            "static_select",
            "external_select",
            "conversations_select",
            "users_select",
            "channels_select",
            "overflow",
            "datepicker",
        ]
        if payload.get("actions"):
            action_type = payload["actions"][0].get("type")
            if action_type in supported:
                return True
            elif action_type:
                logger.warning(
                    "Received input from a Slack interactive component of type "
                    f"'{payload['actions'][0]['type']}', for which payload parsing is not yet supported."
                )
        return False

    @staticmethod
    def _get_interactive_response(action: Dict) -> Optional[Text]:
        """Parse the payload for the response value."""

        if action["type"] == "button":
            return action.get("value")
        elif action["type"] == "select":
            return action.get("selected_options", [{}])[0].get("value")
        elif action["type"] == "static_select":
            return action.get("selected_option", {}).get("value")
        elif action["type"] == "external_select":
            return action.get("selected_option", {}).get("value")
        elif action["type"] == "conversations_select":
            return action.get("selected_conversation")
        elif action["type"] == "users_select":
            return action.get("selected_user")
        elif action["type"] == "channels_select":
            return action.get("selected_channel")
        elif action["type"] == "overflow":
            return action.get("selected_option", {}).get("value")
        elif action["type"] == "datepicker":
            return action.get("selected_date")

    async def process_message(
        self,
        request: Request,
        on_new_message: Callable[[UserMessage], Awaitable[Any]],
        text,
        sender_id: Optional[Text],
        metadata: Optional[Dict],
    ) -> Any:
        """Slack retries to post messages up to 3 times based on
        failure conditions defined here:
        https://api.slack.com/events-api#failure_conditions
        """
        retry_reason = request.headers.get(self.retry_reason_header)
        retry_count = request.headers.get(self.retry_num_header)
        if retry_count and retry_reason in self.errors_ignore_retry:
            logger.warning(
                f"Received retry #{retry_count} request from slack"
                f" due to {retry_reason}."
            )

            return response.text(None, status=201, headers={"X-Slack-No-Retry": 1})

        if metadata is not None:
            output_channel = metadata.get("out_channel")
        else:
            output_channel = None

        try:
            user_msg = UserMessage(
                text,
                self.get_output_channel(output_channel),
                sender_id,
                input_channel=self.name(),
                metadata=metadata,
            )

            await on_new_message(user_msg)
        except Exception as e:
            logger.error(f"Exception when trying to handle message.{e}")
            logger.error(str(e), exc_info=True)

        return response.text("")

    def get_metadata(self, request: Request) -> Dict[Text, Any]:
        

        content_type = request.headers.get("content-type")

        # Slack API sends either a JSON-encoded or a URL-encoded body depending on the content

        if content_type == "application/json":

            # if JSON-encoded message is received

            slack_event = request.json

            event = slack_event.get("event", {})

            thread_id = event.get("thread_ts", event.get("ts"))

            users = (

                slack_event.get("authed_users", [])

                if "authed_users" in slack_event

                else slack_event.get("authorizations")[0].get("user_id")

            )

            return {

                "out_channel": event.get("channel"),

                "thread_id": thread_id,

                "users": users,

            }

        if content_type == "application/x-www-form-urlencoded":

            # if URL-encoded message is received

            output = request.form

            payload = json.loads(output["payload"][0])

            message = payload.get("message", {})

            thread_id = message.get("thread_ts", message.get("ts"))

            return {

                "out_channel": payload.get("channel", {}).get("id"),

                "thread_id": thread_id,

                "users": payload.get("user", {}).get("id"),

            }

        return {}
   

    def blueprint(
        self, on_new_message: Callable[[UserMessage], Awaitable[Any]]
    ) -> Blueprint:
        slack_webhook = Blueprint("slack_webhook", __name__)

        @slack_webhook.route("/", methods=["GET"])
        async def health(_: Request) -> HTTPResponse:
            return response.json({"status": "ok"})

        @slack_webhook.route("/webhook", methods=["GET", "POST"])
        async def webhook(request: Request) -> HTTPResponse:
            if request.form:
                output = request.form
                payload = json.loads(output["payload"][0])

                if self._is_interactive_message(payload):
                    sender_id = payload["user"]["id"]
                    text = self._get_interactive_response(payload["actions"][0])
                    if text is not None:
                        metadata = self.get_metadata(request)
                        return await self.process_message(
                            request, on_new_message, text, sender_id, metadata
                        )
                    elif payload["actions"][0]["type"] == "button":
                        # link buttons don't have "value", don't send their clicks to bot
                        return response.text("User clicked link button")
                return response.text(
                    "The input message could not be processed.", status=500
                )

            elif request.json:
                output = request.json
                event = output.get("event", {})
                user_message = event.get("text", "")
                sender_id = event.get("user", "")
                metadata = self.get_metadata(request)

                if "challenge" in output:
                    return response.json(output.get("challenge"))

                elif self._is_user_message(output) and self._is_supported_channel(
                    output, metadata
                ):
                    return await self.process_message(
                        request,
                        on_new_message,
                        text=self._sanitize_user_message(
                            user_message, metadata["users"]
                        ),
                        sender_id=sender_id,
                        metadata=metadata,
                    )
                else:
                    logger.warning(
                        f"Received message on unsupported channel: {metadata['out_channel']}"
                    )

            return response.text("Bot message delivered.")

        return slack_webhook

    def _is_supported_channel(self, slack_event: Dict, metadata: Dict) -> bool:
        return (
            self._is_direct_message(slack_event)
            or self._is_app_mention(slack_event)
            or metadata["out_channel"] == self.slack_channel
        )

    def get_output_channel(self, channel: Optional[Text] = None) -> OutputChannel:
        channel = channel or self.slack_channel
        return SlackBot(self.slack_token, channel)

    def set_output_channel(self, channel: Text) -> None:
        self.slack_channel = channel


In [24]:
class ActionCheckLocation(Action):

    def name(self):
        return 'action_chklocation'

    def run(self, dispatcher, tracker, domain):
        loc = tracker.get_slot('location')
        
        cities=['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune',
        'Agra', 'Ajmer', 'Aligarh', 'Amravati', 'Amritsar', 'Asansol', 'Aurangabad', 'Bareilly',
        'Belgaum', 'Bhavnagar', 'Bhiwandi', 'Bhopal', 'Bhubaneswar', 'Bikaner', 'Bokaro Steel City',
        'Chandigarh', 'Coimbatore', 'Cuttack', 'Dehradun', 'Dhanbad', 'Durg-Bhilai Nagar', 'Durgapur',
        'Erode', 'Faridabad', 'Firozabad',    'Ghaziabad', 'Gorakhpur', 'Gulbarga', 'Guntur', 'Gurgaon',
        'Guwahati', 'Gwalior', 'Hubli-Dharwad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jammu', 'Jamnagar',
        'Jamshedpur', 'Jhansi', 'Jodhpur', 'Kannur', 'Kanpur', 'Kakinada', 'Kochi', 'Kottayam', 'Kolhapur', 'Kollam',
        'Kota', 'Kozhikode', 'Kurnool', 'Lucknow', 'Ludhiana', 'Madurai', 'Malappuram', 'Mathura', 'Goa', 'Mangalore',
        'Meerut', 'Moradabad', 'Mysore', 'Nagpur', 'Nanded', 'Nashik', 'Nellore', 'Noida', 'Palakkad', 'Patna',
        'Pondicherry', 'Prayagraj', 'Raipur', 'Rajkot', 'Rajahmundry', 'Ranchi', 'Rourkela', 'Salem', 'Sangli',
        'Siliguri', 'Solapur', 'Srinagar', 'Sultanpur', 'Surat', 'Thiruvananthapuram', 'Thrissur', 'Tiruchirappalli',
        'Tirunelveli', 'Tiruppur', 'Ujjain', 'Bijapur', 'Vadodara', 'Varanasi', 'Vasai-Virar City', 'Vijayawada']
        
        cities_lower=[x.lower() for x in cities]
        
        if loc.lower() not in cities_lower:
            dispatcher.utter_message("Sorry, we don’t operate in this city. Can you please specify some other location")
        return

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_sdk import Action
from rasa_sdk.events import SlotSet
import zomatopy
import json

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


class ActionSearchRestaurants(Action):
    def name(self):
        return 'action_search_restaurants'
        
    def run(self, dispatcher, tracker, domain):        
        config={"user_key":"bc4e4a608fc7c3eb63e9f45f80c01fa9"}
        zomato = zomatopy.initialize_app(config)

        loc = tracker.get_slot('location')
        cuisine = tracker.get_slot('cuisine')
        price = tracker.get_slot('price')
        #price='high'
        
        location_detail=zomato.get_location(loc, 1)
        d1 = json.loads(location_detail)
        lat=d1["location_suggestions"][0]["latitude"]
        lon=d1["location_suggestions"][0]["longitude"]
        cuisines_dict={'chinese':25,'italian':55,'north indian':50,'south indian':85,'american':1,'mexican':73}
        price_dict = {'low':1,'medium':2,'high':3}
        results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 50000)
        
        d = json.loads(results)
        count = 0
        response="Showing you top rated restaurants:"

        if d['results_found'] == 0:
            response= "No restaurant found for your criteria"
            dispatcher.utter_message(response)
        else:           
            for restaurant in sorted(d['restaurants'], key=lambda x: x['restaurant']['user_rating']['aggregate_rating'], reverse=True): 
                #Getting Top 10 restaurants for chatbot response
                if((price_dict.get(price) == 1) and (int(restaurant['restaurant']['average_cost_for_two']) < 300) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"."
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1
                elif((price_dict.get(price) == 2) and (int(restaurant['restaurant']['average_cost_for_two']) >= 300) and (int(restaurant['restaurant']['average_cost_for_two']) <= 700) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1                        
                elif((price_dict.get(price) == 3) and (int(restaurant['restaurant']['average_cost_for_two']) > 700) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1           
                if(count==5):
                    dispatcher.utter_message(response)
        if(count<5 and count>0):
            dispatcher.utter_message(response)
        if(count==0):
            response = "Sorry, No results found for your criteria. Would you like to search for some other restaurants?"
            dispatcher.utter_message(response)
        return [SlotSet('emailbody',response)]

class ActionSendEmail(Action):

    def name(self):
        return 'action_sendemail'

    def run(self, dispatcher, tracker, domain):
        from_user = 'baalebail.udemy@gmail.com'
        to_user = tracker.get_slot('email')
        password = 'Epsilon19'
        server = smtplib.SMTP('smtp.gmail.com',587)
        server.starttls()
        server.login(from_user, password)
        subject = 'Hello from chatbox'
        msg = MIMEMultipart()
        msg['From'] = from_user
        msg['TO'] = to_user
        msg['Subject'] = subject
        body = tracker.get_slot('emailbody')
        msg.attach(MIMEText(body,'plain'))
        text = msg.as_string()
        server.sendmail(from_user,to_user,text)
        server.close()
        
class ActionCheckLocation(Action):

    def name(self):
        return 'action_chklocation'

    def run(self, dispatcher, tracker, domain):
        loc = tracker.get_slot('location')
        
        cities=['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune',
        'Agra', 'Ajmer', 'Aligarh', 'Amravati', 'Amritsar', 'Asansol', 'Aurangabad', 'Bareilly',
        'Belgaum', 'Bhavnagar', 'Bhiwandi', 'Bhopal', 'Bhubaneswar', 'Bikaner', 'Bokaro Steel City',
        'Chandigarh', 'Coimbatore', 'Cuttack', 'Dehradun', 'Dhanbad', 'Durg-Bhilai Nagar', 'Durgapur',
        'Erode', 'Faridabad', 'Firozabad',    'Ghaziabad', 'Gorakhpur', 'Gulbarga', 'Guntur', 'Gurgaon',
        'Guwahati', 'Gwalior', 'Hubli-Dharwad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jammu', 'Jamnagar',
        'Jamshedpur', 'Jhansi', 'Jodhpur', 'Kannur', 'Kanpur', 'Kakinada', 'Kochi', 'Kottayam', 'Kolhapur', 'Kollam',
        'Kota', 'Kozhikode', 'Kurnool', 'Lucknow', 'Ludhiana', 'Madurai', 'Malappuram', 'Mathura', 'Goa', 'Mangalore',
        'Meerut', 'Moradabad', 'Mysore', 'Nagpur', 'Nanded', 'Nashik', 'Nellore', 'Noida', 'Palakkad', 'Patna',
        'Pondicherry', 'Prayagraj', 'Raipur', 'Rajkot', 'Rajahmundry', 'Ranchi', 'Rourkela', 'Salem', 'Sangli',
        'Siliguri', 'Solapur', 'Srinagar', 'Sultanpur', 'Surat', 'Thiruvananthapuram', 'Thrissur', 'Tiruchirappalli',
        'Tirunelveli', 'Tiruppur', 'Ujjain', 'Bijapur', 'Vadodara', 'Varanasi', 'Vasai-Virar City', 'Vijayawada']
        
        cities_lower=[x.lower() for x in cities]
        
        if loc.lower() not in cities_lower:
            
            dispatcher.utter_message("Sorry, we don’t operate in this city. Can you please specify some other location")
        return 

In [11]:
d['restaurants']

[{'restaurant': {'R': {'has_menu_status': {'delivery': 1, 'takeaway': -1},
    'res_id': 51705,
    'is_grocery_store': False},
   'apikey': 'bc4e4a608fc7c3eb63e9f45f80c01fa9',
   'id': '51705',
   'name': 'Toit',
   'url': 'https://www.zomato.com/bangalore/toit-indiranagar?utm_source=api_basic_user&utm_medium=api&utm_campaign=v2.1',
   'location': {'address': '298, 100 Feet Road, Namma Metro Pillar 62, Indiranagar, Bangalore',
    'locality': 'Indiranagar',
    'city': 'Bangalore',
    'city_id': 4,
    'latitude': '12.9788880980',
    'longitude': '77.6409219950',
    'zipcode': '0',
    'country_id': 1,
    'locality_verbose': 'Indiranagar, Bangalore'},
   'switch_to_order_menu': 0,
   'cuisines': 'Italian, American, Modern Indian',
   'timings': '12noon – 12midnight (Mon-Tue),12noon – 1am (Wed-Fri),8:30am – 1am (Sat-Sun)',
   'average_cost_for_two': 2000,
   'price_range': 4,
   'currency': 'Rs.',
   'highlights': ['Delivery',
    'Lunch',
    'Credit Card',
    'Debit Card',
    '

In [19]:
count=0
response="Showing you top rated restaurants:"+"\n"
for restaurant in sorted(d['restaurants'], key=lambda x: x['restaurant']['user_rating']['aggregate_rating'], reverse=True):
    #Getting Top 10 restaurants for chatbot response
    print(restaurant['restaurant']['average_cost_for_two'])
    if((price_dict.get(price) == 1) and (int(restaurant['restaurant']['average_cost_for_two']) < 3000) and (count < 10)):
        
        response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"."
        response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
        print(response)           

1600
Showing you top rated restaurants:
Fenny's Lounge And Kitchen in 115, 3rd Floor, Opposite Raheja Arcade, Koramangala 7th Block, Bangalore has been rated 4.6. And the average price for two people is: 1600

1800
Showing you top rated restaurants:
Fenny's Lounge And Kitchen in 115, 3rd Floor, Opposite Raheja Arcade, Koramangala 7th Block, Bangalore has been rated 4.6. And the average price for two people is: 1600
Olive Bar And Kitchen in 16, Wood Street, Ashok Nagar, Richmond Road, Bangalore has been rated 4.6. And the average price for two people is: 1800

1600
Showing you top rated restaurants:
Fenny's Lounge And Kitchen in 115, 3rd Floor, Opposite Raheja Arcade, Koramangala 7th Block, Bangalore has been rated 4.6. And the average price for two people is: 1600
Olive Bar And Kitchen in 16, Wood Street, Ashok Nagar, Richmond Road, Bangalore has been rated 4.6. And the average price for two people is: 1800
Smoke House Deli in Plot 1209, Ward 72, HAL 2nd Stage, Doopanahalli, Indiranaga

In [22]:
response=""
if d['results_found'] == 0:
    response= "no results"
else:
    for restaurant in d['restaurants']:
        response=response+ "Found "+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"

print(response)
      
      

SyntaxError: invalid syntax (<ipython-input-22-3257870d9fea>, line 25)

In [20]:
response="Showing you top rated restaurants:"+"\n"
response

'Showing you top rated restaurants:\n'

In [8]:
if d['results_found'] == 0:
            response= "No restaurant found for your criteria"
            dispatcher.utter_message(response)
            print(response)
else:           
    for restaurant in sorted(d['restaurants'], key=lambda x: x['restaurant']['user_rating']['aggregate_rating'], reverse=True): 
    #Getting Top 10 restaurants for chatbot response
            if((price_dict.get(price) == 1) and (restaurant['restaurant']['average_cost_for_two'] < 300) and (count < 10)):
                response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"."
                response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                count = count + 1
            elif((price_dict.get(price) == 2) and (restaurant['restaurant']['average_cost_for_two'] >= 300) and (restaurant['restaurant']['average_cost_for_two'] <= 700) and (count < 10)):
                response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                count = count + 1                        
            elif((price_dict.get(price) == 3) and (restaurant['restaurant']['average_cost_for_two'] > 700) and (count < 10)):
                response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                count = count + 1           
        if(count==5):
                dispatcher.utter_message(response)
                 print(response)

Found Byg Brewski Brewing Company in Opposite MK Retail, Sarjapur Road, Bangalore has been rated 4.4
Found Toit in 298, 100 Feet Road, Namma Metro Pillar 62, Indiranagar, Bangalore has been rated 4.4
Found Dyu Art Cafe in 23, KHB - Mig Colony, Koramangala 8th Block, Bangalore has been rated 4.2
Found Byg Brewski Brewing Company in 22/123, Byrathi Village, Bidarahalli Hobli, Hennur, Bangalore has been rated 4.2
Found The Hole In The Wall Cafe in 4, 8th Main Road, Koramangala 4th Block, Bangalore has been rated 4.4



In [7]:
pprint.pprint(d)

{'restaurants': [{'restaurant': {'R': {'has_menu_status': {'delivery': -1,
                                                           'takeaway': -1},
                                       'is_grocery_store': False,
                                       'res_id': 58882},
                                 'all_reviews': {'reviews': [{'review': []},
                                                             {'review': []},
                                                             {'review': []},
                                                             {'review': []},
                                                             {'review': []}]},
                                 'all_reviews_count': 16905,
                                 'apikey': 'bc4e4a608fc7c3eb63e9f45f80c01fa9',
                                 'average_cost_for_two': 1600,
                                 'book_again_url': '',
                                 'book_form_web_view_url': '',
                  

In [17]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.message import EmailMessage

# Open the plain text file whose name is in textfile for reading.
#with open(MailBody.txt) as fp:
    # Create a text/plain message
msg = EmailMessage()
#msg.set_content(fp.read())

me = 'praveen.baalebail@gmail.com'
you = 'urspraveenb@gmail.com'
msg['Subject'] = 'chatbot message'
msg['From'] = me
msg['To'] = you

# Send the message via our own SMTP server.
s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [21]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
mail_content = '''Hello,
This is a simple mail. There is only text, no attachments are there The mail is sent using Python SMTP library.
Thank You'''
#The mail addresses and password
sender_address = 'baalebail.udemy@gmail.com'
sender_pass = 'Epsilon19'
receiver_address = 'urspraveenb@gmail.com'
#Setup the MIME
message = MIMEMultipart()
message['From'] = sender_address
message['To'] = receiver_address
message['Subject'] = 'A test mail sent by Python. It has an attachment.'   #The subject line
#The body and the attachments for the mail
message.attach(MIMEText(mail_content, 'plain'))
#Create SMTP session for sending the mail
session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
session.starttls() #enable security
session.login(sender_address, sender_pass) #login with mail_id and password
text = message.as_string()
session.sendmail(sender_address, receiver_address, text)
session.quit()
print('Mail Sent')

Mail Sent


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_sdk import Action
from rasa_sdk.events import SlotSet
import zomatopy
import json

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


class ActionSearchRestaurants(Action):
    def name(self):
        return 'action_search_restaurants'
        
    def run(self, dispatcher, tracker, domain):        
        config={"user_key":"bc4e4a608fc7c3eb63e9f45f80c01fa9"}
        zomato = zomatopy.initialize_app(config)

        loc = tracker.get_slot('location')
        cuisine = tracker.get_slot('cuisine')
        price = tracker.get_slot('price')
        #price='high'
        
        location_detail=zomato.get_location(loc, 1)
        d1 = json.loads(location_detail)
        lat=d1["location_suggestions"][0]["latitude"]
        lon=d1["location_suggestions"][0]["longitude"]
        cuisines_dict={'chinese':25,'italian':55,'north indian':50,'south indian':85,'american':1,'mexican':73}
        price_dict = {'low':1,'medium':2,'high':3}
        results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 50000)
        
        d = json.loads(results)
        count = 0
        response="Showing you top rated restaurants:"

        if d['results_found'] == 0:
            response= "No restaurant found for your criteria"
            dispatcher.utter_message(response)
        else:           
            for restaurant in sorted(d['restaurants'], key=lambda x: x['restaurant']['user_rating']['aggregate_rating'], reverse=True): 
                #Getting Top 10 restaurants for chatbot response
                if((price_dict.get(price) == 1) and (int(restaurant['restaurant']['average_cost_for_two']) < 300) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"."
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1
                elif((price_dict.get(price) == 2) and (int(restaurant['restaurant']['average_cost_for_two']) >= 300) and (int(restaurant['restaurant']['average_cost_for_two']) <= 700) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1                        
                elif((price_dict.get(price) == 3) and (int(restaurant['restaurant']['average_cost_for_two']) > 700) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1           
                if(count==5):
                    dispatcher.utter_message(response)
        if(count<5 and count>0):
            dispatcher.utter_message(response)
        if(count==0):
            response = "Sorry, No results found for your criteria. Would you like to search for some other restaurants?"
            dispatcher.utter_message(response)
        return [SlotSet('emailbody',response)]

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa_sdk import Action
from rasa_sdk.events import SlotSet
import zomatopy
import json

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


class ActionSearchRestaurants(Action):
    def name(self):
        return 'action_search_restaurants'
        
    def run(self, dispatcher, tracker, domain):
        count = 0
        config={"user_key":"bc4e4a608fc7c3eb63e9f45f80c01fa9"}
        zomato = zomatopy.initialize_app(config)

        loc = tracker.get_slot('location')
        cuisine = tracker.get_slot('cuisine')
        #price = tracker.get_slot('price')
        price='high'
        
        location_detail=zomato.get_location(loc, 1)
        d1 = json.loads(location_detail)
        lat=d1["location_suggestions"][0]["latitude"]
        lon=d1["location_suggestions"][0]["longitude"]
        cuisines_dict={'chinese':25,'italian':55,'north indian':50,'south indian':85,'american':1,'mexican':73}
        price_dict = {'low':1,'medium':2,'high':3}
        results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 50000)
        
        d = json.loads(results)
        response="Showing you top rated restaurants:"

        if d['results_found'] == 0:
            response= "No restaurant found for your criteria"
            dispatcher.utter_message(response)
        else:           
            for restaurant in sorted(d['restaurants'], key=lambda x: x['restaurant']['user_rating']['aggregate_rating'], reverse=True): 
                #Getting Top 10 restaurants for chatbot response
                if((price_dict.get(price) == 1) and (int(restaurant['restaurant']['average_cost_for_two']) < 300) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"."
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1
                elif((price_dict.get(price) == 2) and (int(restaurant['restaurant']['average_cost_for_two']) >= 300) and (int(restaurant['restaurant']['average_cost_for_two']) <= 700) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1                        
                elif((price_dict.get(price) == 3) and (int(restaurant['restaurant']['average_cost_for_two']) > 700) and (count < 10)):
                    response=response+ restaurant['restaurant']['name']+ " in "+ restaurant['restaurant']['location']['address']+ " has been rated "+ restaurant['restaurant']['user_rating']['aggregate_rating']+"\n"
                    response=response+" And the average price for two people is: "+ str(restaurant['restaurant']['average_cost_for_two'])+"\n"
                    count = count + 1           
                if(count==5):
                    dispatcher.utter_message(response)
        if(count<5 and count>0):
            dispatcher.utter_message(response)
        if(count==0):
            response = "Sorry, No results found for your criteria. Would you like to search for some other restaurants?"
            dispatcher.utter_message(response)
        return [SlotSet('emailbody',response)]